In [51]:
import pandas as pd
import requests
from io import StringIO

from nst_scraper import nst_on_ice_scraper

pd.set_option('display.max_columns', None)


In [52]:
player_stats_df = nst_on_ice_scraper(fromseason=20232024, thruseason=20232024, startdate='', enddate='')
player_stats_df.head()


,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,DAL,D,82,1403.833333,2,14,6,8,16,26.23,99,2.02,3.72,245,159,67,6,2,15,24,12,12,0,0,0,28,14,75,92,86,0,0,-
1,Jeff Carter,PIT,C,72,664.900000,5,3,1,2,8,61.54,65,7.69,5.86,122,96,61,31,6,13,10,5,5,0,0,3,16,16,75,32,25,169,87,66.02
2,Zach Parise,COL,L,30,340.616667,4,3,1,2,7,70.00,43,9.30,4.56,68,54,41,23,4,11,6,3,3,0,0,2,4,10,20,22,23,11,21,34.38
3,Brent Burns,CAR,D,82,1276.683333,5,12,5,7,17,26.56,134,3.73,4.75,417,225,96,10,6,43,18,9,9,0,0,5,43,30,34,33,72,0,0,-
4,Corey Perry,"CHI, EDM",R,54,600.266667,9,7,4,3,16,53.33,61,14.75,7.57,114,92,71,37,4,13,46,17,13,4,0,14,17,15,42,66,17,0,1,0.00


In [53]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20232024, thruseason=20232024, startdate='', enddate='', pos='g')
goalie_stats_df.head()


,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,40,1758.750000,802,738,64,0.920,2.18,5.40,65.04,170,142,28,0.835,0.96,2.91,196,172,24,0.878,0.82,-2.53,394,382,12,0.970,0.41,0.07,49,125,37.33,21.02
1,Jonathan Quick,NYR,27,1283.333333,618,560,58,0.906,2.71,-4.52,58.21,164,134,30,0.817,1.40,-0.19,170,149,21,0.876,0.98,-2.38,254,247,7,0.972,0.33,0.78,40,124,33.47,18.93
2,James Reimer,DET,25,1094.083333,551,496,55,0.900,3.02,-7.32,47.93,130,100,30,0.769,1.65,-6.37,140,122,18,0.871,0.99,-2.67,253,246,7,0.972,0.38,0.75,37,89,36.94,19.56
3,Semyon Varlamov,NYI,28,1301.083333,646,599,47,0.927,2.17,8.90,58.18,161,137,24,0.851,1.11,5.27,150,137,13,0.913,0.60,3.43,302,292,10,0.967,0.46,-0.75,46,143,34.86,22.83
4,Jacob Markstrom,CGY,48,2268.283333,1063,973,90,0.915,2.38,1.99,101.53,302,262,40,0.868,1.06,14.90,263,230,33,0.875,0.87,-4.19,449,432,17,0.962,0.45,-3.24,78,203,33.56,22.72


In [54]:
class Lineup:
    def __init__(self, name):
        self.name = name
        self.forwards = [None] * 12  # 12 Forward Slots
        self.defense = [None] * 6    # 6 Defense Slots
        self.goalies = [None] * 2    # 2 Goalie Slots

    def add_forward(self, player, slot):
        allowed_positions = {'L', 'C', 'R'}
        if player.position not in allowed_positions:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to forwards. "
                f"Allowed positions: {', '.join(allowed_positions)}."
            )
        if 0 <= slot < 12:
            if self.forwards[slot] is not None:
                print(f"Warning: Slot {slot + 1} in forwards is already occupied by '{self.forwards[slot].name}'. Overwriting.")
            self.forwards[slot] = player
            print(f"Added player '{player.name}' to forward slot {slot + 1}.")
        else:
            raise IndexError("Forward slot must be between 0 and 11.")

    def add_defense(self, player, slot):
        allowed_position = 'D'
        if player.position != allowed_position:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to defense. "
                f"Allowed position: '{allowed_position}'."
            )
        if 0 <= slot < 6:
            if self.defense[slot] is not None:
                print(f"Warning: Slot {slot + 1} in defense is already occupied by '{self.defense[slot].name}'. Overwriting.")
            self.defense[slot] = player
            print(f"Added player '{player.name}' to defense slot {slot + 1}.")
        else:
            raise IndexError("Defense slot must be between 0 and 5.")

    def set_goalie(self, player, slot):
        allowed_position = 'G'
        if player.position != allowed_position:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to goalies. "
                f"Allowed position: '{allowed_position}'."
            )
        if 0 <= slot < 2:
            if self.goalies[slot] is not None:
                print(f"Warning: Goalie slot {slot + 1} is already occupied by '{self.goalies[slot].name}'. Overwriting.")
            self.goalies[slot] = player
            print(f"Added goalie '{player.name}' to goalie slot {slot + 1}.")
        else:
            print(f"Cannot assign goalie to slot {slot}. Maximum of two goalies allowed.")

    def display_lineup(self):
        print(f"Lineup: {self.name}")
        print("Forwards:")
        for idx, player in enumerate(self.forwards):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
        print("Defense:")
        for idx, player in enumerate(self.defense):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
        print("Goalies:")
        for idx, player in enumerate(self.goalies):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
    def to_dataframe(self):
        """
        Converts the lineup into a pandas DataFrame.

        Returns:
            pd.DataFrame: A DataFrame containing forwards, defensemen, and goalies with their respective slots.
        """
        import pandas as pd

        # Prepare data for forwards
        forwards_data = [{
            'Position': 'F',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.forwards)]

        # Prepare data for defensemen
        defense_data = [{
            'Position': 'D',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.defense)]

        # Prepare data for goalies
        goalies_data = [{
            'Position': 'G',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.goalies)]

        # Combine all data
        combined_data = forwards_data + defense_data + goalies_data

        # Create DataFrame
        df = pd.DataFrame(combined_data)

        return df
    def to_transposed_dataframe(self):
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        (e.g., 'Forward1', 'Defense1', 'Goalie1') and the row contains the corresponding player names.

        Returns:
            pd.DataFrame: A transposed DataFrame with position-slot as columns and player names as row values.
        """
        import pandas as pd

        # Generate the original DataFrame using the existing method
        df = self.to_dataframe()

        # Initialize an empty dictionary to hold the transposed data
        transposed_data = {}

        # Iterate over each row in the original DataFrame
        for _, row in df.iterrows():
            position = row['Position']
            slot = row['Slot']
            player = row['Player']
            # Create a new column name by combining position and slot number
            column_name = f"{position}{slot}"
            transposed_data[column_name] = player

        # Create a new DataFrame with a single row using the transposed data
        transposed_df = pd.DataFrame([transposed_data])

        return transposed_df

In [55]:
# Define the Player class
class Player:
    def __init__(self, name, team, position, player_id=None):
        """
        Initializes a new Player instance.

        :param name: Full name of the player
        :param team: Team the player belongs to
        :param position: Position of the player (e.g., C, LW, RW, D, G)
        :param player_id: Unique identifier for the player (optional)
        """
        self.player_id = player_id
        self.name = name
        self.team = team
        self.position = position

    def __str__(self):
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self):
        """
        Converts the Player instance into a dictionary.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position
        }


In [56]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=row['position']
        # player_id is not set initially
    )
    player_list.append(player)

In [57]:
# Creating Player instances from the player_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position='G'
        # player_id is not set initially
    )
    goalie_list.append(player)

In [58]:
# Convert player list to DataFrame
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position
} for player in player_list]).head()

,name,team,position
0,Ryan Suter,DAL,D
1,Jeff Carter,PIT,C
2,Zach Parise,COL,L
3,Brent Burns,CAR,D
4,Corey Perry,"CHI, EDM",R


In [59]:
# Convert player list to DataFrame
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,DET,G
3,Semyon Varlamov,NYI,G
4,Jacob Markstrom,CGY,G


In [60]:
def create_lineup(team):
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

my_lineup = create_lineup('TOR')

Skipping player 'Mark Giordano': Cannot add player 'Mark Giordano' with position 'D' to forwards. Allowed positions: L, R, C.
Added player 'Ryan Reaves' to forward slot 1.
Skipping player 'TJ Brodie': Cannot add player 'TJ Brodie' with position 'D' to forwards. Allowed positions: L, R, C.
Added player 'John Tavares' to forward slot 2.
Added player 'Calle Jarnkrok' to forward slot 3.
Skipping player 'John Klingberg': Cannot add player 'John Klingberg' with position 'D' to forwards. Allowed positions: L, R, C.
Skipping player 'Morgan Rielly': Cannot add player 'Morgan Rielly' with position 'D' to forwards. Allowed positions: L, R, C.
Skipping player 'Jake McCabe': Cannot add player 'Jake McCabe' with position 'D' to forwards. Allowed positions: L, R, C.
Added player 'Tyler Bertuzzi' to forward slot 4.
Added player 'Max Domi' to forward slot 5.
Added player 'William Nylander' to forward slot 6.
Added player 'Mitch Marner' to forward slot 7.
Added player 'Auston Matthews' to forward slot 8

In [61]:
# Convert the lineup to a transposed DataFrame
transposed_lineup_df = my_lineup.to_transposed_dataframe()

# Display the transposed DataFrame
transposed_lineup_df

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,D1,D2,D3,D4,D5,D6,G1,G2
0,Ryan Reaves,John Tavares,Calle Jarnkrok,Tyler Bertuzzi,Max Domi,William Nylander,Mitch Marner,Auston Matthews,Noah Gregor,David Kampf,Pontus Holmberg,Nicholas Robertson,Mark Giordano,TJ Brodie,John Klingberg,Morgan Rielly,Jake McCabe,Maxime Lajoie,Martin Jones,Ilya Samsonov


In [10]:
from pbp import get_team_roster

get_team_roster('TOR', 20232024)

{'forwards': [{'id': 8481720,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/TOR/8481720.png',
   'firstName': {'default': 'Nick',
    'cs': 'Nicholas',
    'de': 'Nicholas',
    'es': 'Nicholas',
    'fi': 'Nicholas',
    'sk': 'Nicholas',
    'sv': 'Nicholas'},
   'lastName': {'default': 'Abruzzese'},
   'sweaterNumber': 26,
   'positionCode': 'C',
   'shootsCatches': 'L',
   'heightInInches': 71,
   'weightInPounds': 180,
   'heightInCentimeters': 180,
   'weightInKilograms': 82,
   'birthDate': '1999-06-04',
   'birthCity': {'default': 'Slate Hill'},
   'birthCountry': 'USA',
   'birthStateProvince': {'default': 'NY'}},
  {'id': 8480980,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/TOR/8480980.png',
   'firstName': {'default': 'Connor'},
   'lastName': {'default': 'Dewar'},
   'sweaterNumber': 24,
   'positionCode': 'C',
   'shootsCatches': 'L',
   'heightInInches': 70,
   'weightInPounds': 183,
   'heightInCentimeters': 178,
   'weightInKilograms': 83,
   '

In [13]:
from pbp_utils import get_matchup_games

games = get_matchup_games('2024-11-11', '2024-11-17')
pd.DataFrame({
    'game_id': games['game_ids']['id'],
    'date': games['game_ids']['date']
}).head()


,game_id,date
0,2024020239,2024-11-11
1,2024020240,2024-11-11
2,2024020241,2024-11-11
3,2024020242,2024-11-11
4,2024020243,2024-11-11


In [8]:
from game_utils import get_game_boxscore, display_boxscore

temp_data = get_game_boxscore(2024020255)

In [9]:
away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(temp_data)
away_skaters


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8477960,9,A. Kempe,R,2,0,2,0,0,1,0,4,0.000000,19:11,1,23,0,0,Away
1,8471685,11,A. Kopitar,C,0,2,2,0,0,0,0,1,0.769231,19:29,0,23,0,0,Away
2,8479675,12,T. Moore,L,0,2,2,0,0,1,0,0,0.000000,18:52,1,24,0,0,Away
3,8482155,14,A. Laferriere,R,0,0,0,-3,0,2,0,1,0.000000,15:46,0,22,0,1,Away
4,8477942,22,K. Fiala,L,0,0,0,-1,2,2,0,1,0.000000,18:00,2,22,2,0,Away
5,8476479,24,P. Danault,C,0,0,0,-1,2,1,0,0,0.636364,18:11,1,25,1,1,Away
6,8480851,26,A. Thomas,C,0,0,0,0,0,1,0,0,0.000000,11:09,1,15,0,0,Away
7,8477998,37,W. Foegele,L,0,0,0,-1,0,2,0,1,0.000000,15:37,1,25,2,0,Away
8,8481732,47,A. Lee,L,0,0,0,-1,0,3,0,2,0.000000,10:43,2,13,0,1,Away
9,8482124,55,Q. Byfield,R,0,0,0,-1,0,1,0,1,0.500000,13:46,0,22,2,0,Away


In [10]:
away_goalies


,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8479496,31,D. Rittich,G,4/5,0/0,0/0,4/5,0.8,1,0,0,0,1,14:41,False,L,5,4,Away
1,8475311,35,D. Kuemper,G,17/19,1/1,0/0,18/20,0.9,2,0,0,0,2,44:32,True,NaN,20,18,Away


In [11]:
home_skaters

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8476391,9,T. Tynan,C,0,0,0,0,0,0,0,0,0.000000,07:39,1,12,1,0,Home
1,8475780,14,C. Wagner,R,0,0,0,0,0,0,0,0,0.000000,04:24,0,10,0,0,Home
2,8480448,17,P. Kelly,C,0,0,0,0,0,3,0,1,0.555556,13:12,2,21,1,0,Home
3,8481186,25,L. O'Connor,R,0,1,1,1,0,1,0,0,0.000000,15:56,0,23,1,0,Home
4,8477492,29,N. MacKinnon,C,0,3,3,3,0,1,0,3,0.272727,24:45,0,27,1,0,Home
5,8479999,37,C. Mittelstadt,C,0,0,0,-1,0,0,0,1,0.428571,18:35,0,22,3,2,Home
6,8481042,51,N. Kovalenko,R,0,0,0,-1,2,2,0,0,0.000000,14:57,0,19,0,0,Home
7,8477476,62,A. Lehkonen,L,1,1,2,2,0,1,0,3,0.000000,23:13,1,25,0,0,Home
8,8483930,82,I. Ivan,C,0,0,0,-1,0,1,0,1,0.000000,16:18,1,19,0,0,Home
9,8485105,85,N. Prishchepov,C,0,0,0,0,0,1,0,1,0.000000,04:23,0,8,0,0,Home


In [14]:
home_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8480382,40,A. Georgiev,G,11/13,2/2,0/0,13/15,0.866667,2,0,0,0,2,60:00,True,W,15,13,Home
1,8481020,60,J. Annunen,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,0,00:00,False,NaN,0,0,Home


In [17]:
lineup2.display_lineup()


Lineup: Lineup 2
Forwards:
  Slot 1: Empty
  Slot 2: Empty
  Slot 3: Empty
  Slot 4: Empty
  Slot 5: Empty
  Slot 6: Empty
  Slot 7: Empty
  Slot 8: Empty
  Slot 9: Empty
  Slot 10: Empty
  Slot 11: Empty
  Slot 12: Empty
Defense:
  Slot 1: Empty
  Slot 2: Empty
  Slot 3: Empty
  Slot 4: Empty
  Slot 5: Empty
  Slot 6: Empty
Goalie: Empty

